In [1]:
from sklearn import datasets
import pandas as pd
from tg.common.ml import batched_training as bt

## Binary classification

In [2]:
from yo_fluq_ds import *

def get_binary_classification_bundle():
    ds = datasets.load_breast_cancer()
    features = pd.DataFrame(ds['data'], columns=ds['feature_names'])
    df = pd.DataFrame(ds['target'], columns=['label'])
    df['split'] = bt.train_display_test_split(df, 0.2, 0.2, 'label')
    bundle = bt.DataBundle(index=df, features=features)
    return bundle

get_binary_classification_bundle().index.head()

,label,split
0,0,display
1,0,train
2,0,train
3,0,train
4,0,train


In [3]:
get_binary_classification_bundle().features.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
from tg.grammar_ru.ml.components.training_task_factory import TaskFactory, Conventions
from tg.common.ml import dft

def get_feature_extractor():
    feature_extractor = (bt.PlainExtractor
                 .build('features')
                 .index('features')
                 .apply(transformer = dft.DataFrameTransformerFactory.default_factory())
                )
    return feature_extractor
    
def get_binary_label_extractor():
    label_extractor = (bt.PlainExtractor
                   .build(Conventions.LabelFrame)
                   .index()
                   .apply(take_columns=['label'], transformer=None)
                  )
    return label_extractor

def test_extractor(extractor, bundle):
    extractor.fit(bundle)
    return extractor.extract(bundle)

db = get_binary_classification_bundle()
idb = bt.IndexedDataBundle(db.index, db)
test_extractor( get_feature_extractor(), idb).head()

ImportError: cannot import name 'TaskFactory' from 'tg.grammar_ru.ml.components.training_task_factory' (/home/yura/Desktop/repos/grammar_ru_private/grammar_ru/tg/grammar_ru/ml/components/training_task_factory/__init__.py)

In [ ]:
test_extractor(get_binary_label_extractor(), idb).head()

In [ ]:
import torch

class ClassificationNetwork(torch.nn.Module):
    def __init__(self, hidden_size, sample):
        super(ClassificationNetwork, self).__init__()
        self.hidden = torch.nn.Linear(sample['features'].shape[1], hidden_size)
        self.output = torch.nn.Linear(hidden_size, sample['label'].shape[1])

    def forward(self, input):
        X = input['features']
        X = torch.tensor(X.astype(float).values).float()
        X = self.hidden(X)
        X = torch.sigmoid(X)
        X = self.output(X)
        X = torch.sigmoid(X)
        return X


In [ ]:
from sklearn.metrics import roc_auc_score
from tg.common import Logger
Logger.disable()

class ClassificationTask(TaskFactory):
    def create_task(self, data, env):
        metrics = bt.MetricPool().add_sklearn(roc_auc_score)
        self.instantiate_default_task(epoch_count=10, batch_size=1000, mini_batch_size=None, metric_pool=metrics)
        self.setup_batcher(data, [get_feature_extractor(), get_binary_label_extractor()])
        self.setup_model(lambda _, sample: ClassificationNetwork(100, sample))
        
task = ClassificationTask()
result = task.run(get_binary_classification_bundle())
pd.DataFrame(result['output']['history']).set_index('iteration').plot()

In [ ]:
from tg.common.ml.batched_training.torch import networks as btn

class ClassificationTask(TaskFactory):
    def create_task(self, data, env):
        metrics = bt.MetricPool().add_sklearn(roc_auc_score)
        self.instantiate_default_task(epoch_count=10, batch_size=1000, mini_batch_size=None, metric_pool=metrics)
        self.setup_batcher(data, [get_feature_extractor(), get_binary_label_extractor()])
        network_factory = btn.FullyConnectedNetwork.Factory([100],output='label').prepend_extraction('features')
        self.setup_model(network_factory)
        
task = ClassificationTask()
result = task.run(get_binary_classification_bundle())
pd.DataFrame(result['output']['history']).set_index('iteration').plot()

## Multilabel classification

In [ ]:
def get_multilabel_classification_bundle():
    ds = datasets.load_iris()
    features = pd.DataFrame(ds['data'], columns=ds['feature_names'])
    df = pd.DataFrame(ds['target_names'][ds['target']], columns = ['label'])
    df['split'] = bt.train_display_test_split(df, 0.2, 0.2, 'label')
    bundle = bt.DataBundle(index=df, features=features)
    return bundle
    
get_multilabel_classification_bundle().index.head()

In [ ]:
def get_multilabel_extractor():
    label_extractor = (bt.PlainExtractor
                   .build(Conventions.LabelFrame)
                   .index()
                   .apply(take_columns=['label'], transformer=dft.DataFrameTransformerFactory.default_factory())
                  )
    return label_extractor

db = get_multilabel_classification_bundle()
idb = bt.IndexedDataBundle(db.index, db)
test_extractor( get_multilabel_extractor(), idb).head()

In [ ]:
class MulticlassMetrics(bt.Metric):
    def __init__(self, add_accuracy = True, add_rating = False):
        self.add_accuracy = add_accuracy
        self.add_rating = add_rating
    
    def get_names(self):
        result = []
        if self.add_accuracy:
            result.append('accuracy')
        if self.add_rating:
            result.append('rating')
        return result
    
    def measure(self, df, _):
        prefix = 'true_label_'
        labels = []
        for c in df.columns:
            if c.startswith(prefix):
                labels.append(c.replace(prefix,''))

        def ustack(df, prefix, cols, name):
            df = df[[prefix+c for c in cols]]
            df.columns=[c for c in cols]
            df = df.unstack().to_frame(name)
            return df

        predicted = ustack(df, 'predicted_label_', labels, 'predicted')
        true = ustack(df, 'true_label_', labels, 'true')
        df = predicted.merge(true, left_index=True, right_index=True).reset_index()
        df.columns=['label','sample','predicted','true']
        df = df.feed(fluq.add_ordering_column('sample',('predicted',False), 'predicted_rating'))

        match = (df.loc[df.predicted_rating==0].set_index('sample').true>0.5)
        rating = df.loc[df.true>0.5].set_index('sample').predicted_rating
        result = []
        if self.add_accuracy:
            result.append(match.mean())
        if self.add_rating:
            result.append(rating.mean())
        return result


class ClassificationTask(TaskFactory):
    def create_task(self, data, env):
        metrics = bt.MetricPool().add(MulticlassMetrics())
        self.instantiate_default_task(epoch_count=20, batch_size=10000, mini_batch_size=None, metric_pool=metrics)
        self.setup_batcher(data, [get_feature_extractor(), get_multilabel_extractor()])
        self.setup_model(lambda _, sample: ClassificationNetwork(20, sample), learning_rate=1)
        
task = ClassificationTask()
result = task.run(get_multilabel_classification_bundle())
pd.DataFrame(result['output']['history']).set_index('iteration').plot()